In [1]:
# импортирование библиотек
import os
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# чтение данных

# путь к БД на вашем компьютере (например, в той же папке, что и тетрадь)
path_to_db_local = r'C:\Users\forea\Downloads\telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

# если путь на вашем компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

#создаём подключение к базе
engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    
# SQL-запрос для получения данных из разных таблиц БД
query = """
    SELECT u.user_id,
           u.lt_day,
           CASE
                WHEN u.lt_day <= 365 THEN 'new'
                WHEN u.lt_day > 365 THEN 'old'
           END as is_new,
           u.age,
           CASE
               WHEN u.gender_segment == 1 THEN 'Female'
               WHEN u.gender_segment == 0 THEN 'Male'
           END AS gender_segment,
           u.os_name,
           u.cpe_type_name,
           l.country AS country,
           l.city AS city,
		   CASE
		       WHEN aseg.title IS NOT NULL THEN substr(aseg.title, 4)
		   END as age_segment,
		   CASE
		       WHEN ts.title IS NOT NULL THEN substr(ts.title, 4)
		   END as traffic_segment,
		   CASE
		       WHEN ls.title IS NOT NULL THEN substr(ls.title, 4)
		   END as lifetime_segment,
           u.nps_score,
           CASE
               WHEN u.nps_score <= 6 THEN 'detractors'
               WHEN u.nps_score > 6 AND u.nps_score <=8 THEN 'passives'
               WHEN u.nps_score >=9 THEN 'promoters'
           END as nps_group
     FROM user AS u
     LEFT OUTER JOIN location AS l ON u.location_id = l.location_id
     LEFT OUTER JOIN age_segment AS aseg ON u.age_gr_id = aseg.age_gr_id
     LEFT OUTER JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
     LEFT OUTER JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id
     WHERE u.gender_segment IS NOT NULL AND aseg.title IS NOT '08 n/a';
    """
# создаём датафрейм по данным запроса
df = pd.read_sql(query, engine)

In [3]:
# оценка результатов выгрузки
df.head

<bound method NDFrame.head of        user_id  lt_day is_new   age gender_segment  os_name cpe_type_name  \
0       A001A2    2320    old  45.0         Female  ANDROID    SMARTPHONE   
1       A001WF    2344    old  53.0           Male  ANDROID    SMARTPHONE   
2       A003Q7     467    old  57.0           Male  ANDROID    SMARTPHONE   
3       A004TB    4190    old  44.0         Female      IOS    SMARTPHONE   
4       A004XT    1163    old  24.0           Male  ANDROID    SMARTPHONE   
...        ...     ...    ...   ...            ...      ...           ...   
501147  ZZZKLD    1249    old  54.0         Female  ANDROID    SMARTPHONE   
501148  ZZZLWY     129    new  31.0           Male  ANDROID    SMARTPHONE   
501149  ZZZQ5F     522    old  36.0           Male  ANDROID    SMARTPHONE   
501150  ZZZQ8E    2936    old  37.0         Female  ANDROID    SMARTPHONE   
501151  ZZZYH0    2471    old  37.0           Male      IOS    SMARTPHONE   

       country          city age_segment traf

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501152 entries, 0 to 501151
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           501152 non-null  object 
 1   lt_day            501152 non-null  int64  
 2   is_new            501152 non-null  object 
 3   age               501152 non-null  float64
 4   gender_segment    501152 non-null  object 
 5   os_name           501152 non-null  object 
 6   cpe_type_name     501152 non-null  object 
 7   country           501152 non-null  object 
 8   city              501152 non-null  object 
 9   age_segment       501152 non-null  object 
 10  traffic_segment   501152 non-null  object 
 11  lifetime_segment  501152 non-null  object 
 12  nps_score         501152 non-null  int64  
 13  nps_group         501152 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.5+ MB


In [5]:
df.to_csv('telecomm_dashboard_data.csv', index = False)

### Ссылки на материалы проекта
Ссылка на дашборд

https://public.tableau.com/app/profile/anna.serikova7712/viz/sumup_project_2_dashboard/NPS_3?publish=yes

Ссылка на презентацию

https://disk.yandex.ru/i/7oTKe-vZHZ6iGA